<a href="https://colab.research.google.com/github/Elena0591/Hello-World/blob/main/Malware_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Importación de librerias 

In [ ]:
import sys
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import random
import json
from pandas.io.json import json_normalize
from scipy import stats
from datetime import datetime

from sklearn import model_selection
import xgboost as xgb 

from sklearn import preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

###Importación del dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/Machine Learning/Lenguaje supervisado/sample_mmp.csv", sep = ";")

###EDA

In [ ]:
df.info(verbose = False )

In [ ]:
df.info()

In [ ]:
df.head(5)

In [ ]:
TARGET = 'HasDetections'

In [ ]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
df.set_index('MachineIdentifier', inplace = True)

In [ ]:
filtro_id = [col1 for col1 in df if col1.find('Identifier')>=0]
filtro_id

In [ ]:
df.drop([ 'CityIdentifier', 'GeoNameIdentifier','LocaleEnglishNameIdentifier', 'IeVerIdentifier','Census_ProcessorManufacturerIdentifier','Census_ProcessorModelIdentifier', 'Census_OSInstallLanguageIdentifier', 'Census_FirmwareManufacturerIdentifier', 'Census_FirmwareVersionIdentifier'], axis=1, inplace=True)

In [ ]:
df.info(verbose = False)

Deixo ID que crec que ens poden donar informació, la resta els elimino pq no aporten res al model. 

In [ ]:
df.describe(include = np.number).T

In [ ]:
df.describe(exclude = np.number).T

In [ ]:
df['PuaMode'].isnull().sum() * 100/500000

In [ ]:
df['PuaMode'].value_counts()

In [ ]:
df.drop(["PuaMode"], axis=1, inplace=True)

Elimino la variable PuaMode pq la columna unique és = 1 i no ens aporta res al model, a part de que té un 99% de nuls. 

In [ ]:
df.info(verbose = False)

In [ ]:
description = df.describe(include = 'all').T

In [ ]:
description.to_excel('df_description.xlsx') #-->paso a a excel per tenir un control de les variables 

###Analisis del TARGET

In [ ]:
df[TARGET].describe()

In [ ]:
df[TARGET].fillna(0.0, inplace=True)

In [ ]:
df[TARGET].head()

Veiem que el TARGET está balancejat i hi ha casi el 50% de maquines amb malware i un altre 50% sense malware


In [ ]:
sns.countplot(data=df, x='HasDetections')

In [ ]:
df['HasDetections'].value_counts(dropna=False)

###Analizar variables categoricas

###Product Name

In [ ]:
df['ProductName'].describe()

La mayoria de maquinas tienen win8defender.

In [ ]:
df['ProductName'].value_counts()

In [ ]:
df['Platform'].describe()

In [ ]:
df['Platform'].value_counts()

In [ ]:
df['Census_ChassisTypeName'].describe()

In [ ]:
df['Census_ChassisTypeName'].value_counts()

In [ ]:
top_chassis_list = df['Census_ChassisTypeName'].value_counts().head(5).index.to_list()

In [ ]:
top_chassis_list.append('Others')

In [ ]:
df['Census_ChassisTypeName'] = pd.Categorical(df['Census_ChassisTypeName'], categories = top_chassis_list )

In [ ]:
df['Census_ChassisTypeName'].value_counts(dropna = False)

In [ ]:
df['Census_ChassisTypeName'].fillna('Others', inplace = True)

In [ ]:
df['Census_ChassisTypeName'].value_counts(normalize = True)

In [ ]:
df.pivot_table (index = 'ProductName', values = TARGET, aggfunc = [len, np.mean])

In [ ]:
df.pivot_table (index = 'Platform', values = TARGET, aggfunc = [len, np.mean])

In [ ]:
df['Census_InternalBatteryType'].describe()

In [ ]:
df['Census_InternalBatteryType'].value_counts(dropna = False)

In [ ]:
top_battery_list = df['Census_InternalBatteryType'].value_counts().head(5).index.to_list()

In [ ]:
top_battery_list.append('Others')

In [ ]:
df['Census_InternalBatteryType'] = pd.Categorical(df['Census_InternalBatteryType'], categories = top_battery_list )

In [ ]:
df['Census_InternalBatteryType'].value_counts(dropna = False)

In [ ]:
df['Census_InternalBatteryType'].fillna('Others', inplace = True)

In [ ]:
df['Census_InternalBatteryType'].value_counts(dropna = False)

In [ ]:
df['Census_OSEdition'].describe()

In [ ]:
df['Census_OSEdition'].value_counts(dropna = False)

In [ ]:
top_edition_list = df['Census_OSEdition'].value_counts().head(5).index.to_list()

In [ ]:
top_edition_list.append('Others')

In [ ]:
df['Census_OSEdition'] = pd.Categorical(df['Census_OSEdition'], categories = top_edition_list )

In [ ]:
df['Census_OSEdition'].value_counts(dropna = False)

In [ ]:
df['Census_OSEdition'].fillna('Others', inplace = True)

In [ ]:
df['Census_OSEdition'].value_counts(dropna = False)

In [ ]:
def plot_cat_values(dataframe, column, target_column):
    plt.figure(figsize=(15,8))
    #en una única grafica de 2x1 en el primer slot
    ax1 = plt.subplot(2,1,1)
    ## Graficamos el conteo de cada uno de los valores
    ax1 = sns.countplot(
        dataframe[column],
        order = list(dataframe[column].unique())
    )
    ax2 = plt.subplot(2,1,2) # share ax1 para que me pinte el axis en el mismo orden
    ## Graficamos la distribución del target sólo para aquellos casos con target > 0 (para que no se chafe el BP) 
    ax2 = sns.boxenplot(
        data = dataframe[dataframe[target_column] > 0], 
        x = column,
        y = target_column,
        order = list(dataframe[column].unique())
    )
    plt.show()

In [ ]:
plot_cat_values(df, 'Census_ChassisTypeName', TARGET)

In [ ]:
plot_cat_values(df, 'Census_InternalBatteryType', TARGET)

In [ ]:
plot_cat_values(df, 'Census_OSEdition', TARGET)

In [ ]:
plot_cat_values(df, 'Platform', TARGET)

In [ ]:
plot_cat_values(df, 'ProductName', TARGET)

In [ ]:
df['Census_OSEdition'].value_counts()

In [ ]:
df['Census_OSSkuName'].value_counts().head(5)

In [ ]:
pd.crosstab(df["Census_OSSkuName"], df["Census_OSEdition"])

In [ ]:
df.drop(["Census_OSSkuName"], axis=1, inplace=True)

In [ ]:
df['Census_OSBranch'].value_counts()

Elimino la variable Census_OSSkuName pq explica el mateix que la Census_OSEdition. 

In [ ]:
def setOthers(dataframe, column, num_values):
    top_categories = dataframe[column].value_counts().head(num_values)
    top_categories_list = top_categories.index.to_list()
    top_categories_list.append('Others')
    dataframe[column] = pd.Categorical(dataframe[column], categories=top_categories_list)
    return dataframe[column].fillna('Others')

In [ ]:
df['Census_OSBranch'] = setOthers(df, 'Census_OSBranch', 6)

In [ ]:
df['SmartScreen'].value_counts()

In [ ]:
df['SmartScreen'] = setOthers(df, 'SmartScreen', 6)

In [ ]:
df['Census_MDC2FormFactor'].value_counts().head(5)

In [ ]:
df['Census_MDC2FormFactor'] = setOthers(df, 'Census_MDC2FormFactor', 6)

In [ ]:
df['Census_ChassisTypeName'].value_counts()

In [ ]:
df['Census_ActivationChannel'].value_counts()

In [ ]:
df['Census_FlightRing'].value_counts()

###LIMPIEZA DE ATRIBUTOS

###Ajuste de tipos

In [ ]:
selec_id = [col1 for col1 in df if col1.find('Identifier')>=0]
selec_id

In [ ]:
df['DefaultBrowsersIdentifier'] = df['DefaultBrowsersIdentifier'].astype(object)

In [ ]:
def transform (dataframe, column):
  column.find('Identifier')
  dataframe[column] = dataframe[column].astype(object)

In [ ]:
df['DefaultBrowsersIdentifier'] = transform(df, 'DefaultBrowsersIdentifier')

In [ ]:
df['DefaultBrowsersIdentifier'].describe()

In [ ]:
df['AVProductStatesIdentifier'] = transform(df, 'AVProductStatesIdentifier')

In [ ]:
df['CountryIdentifier'] = transform(df, 'CountryIdentifier')

In [ ]:
df['Census_OEMNameIdentifier'] = transform(df, 'Census_OEMNameIdentifier')

In [ ]:
df['Census_OEMModelIdentifier'] = transform(df, 'Census_OEMModelIdentifier')

In [ ]:
df['Census_OSUILocaleIdentifier'] = transform(df, 'Census_OSUILocaleIdentifier')

In [ ]:
df['Wdft_RegionIdentifier'] = transform(df, 'Wdft_RegionIdentifier')

He transformado los id en objeto para luego volver a generalos a numericos

In [ ]:
df.info(verbose = True)

###Imputar valores nulos

Variables categoricas

In [ ]:
df.select_dtypes(include = ['object']).isnull().sum() 

In [ ]:
df.drop(["Census_ProcessorClass"], axis=1, inplace=True) #--> elimino pq el 99% de valors que té, són nuls

In [ ]:
df['OsBuildLab'].value_counts(dropna = False).head(10)

In [ ]:
df['OsBuildLab'].fillna(value = "zzzz", inplace = True)

In [ ]:
for i in df.select_dtypes(include = ['object']):
    print(i, df[i].fillna(value = "zzzz", inplace = True))

Variables numeriques

In [ ]:
df.select_dtypes(include = ['number']).isnull().sum() 

In [ ]:
for i in df.select_dtypes(include = ['number']):
    print(i, df[i].fillna(value = -100, inplace = True))

**FREQUENCY ENCODING DE LAS VARIABLES QUE EN LA COLUMNA UNIQUE SON > 100**

In [ ]:
freq_enc = ['AvSigVersion', 'OsBuildLab', 'Census_OSVersion']

In [ ]:
df_AvSigVersion_num = pd.DataFrame(df['AvSigVersion'].value_counts(dropna = False))# variable AvSigVersion

In [ ]:
df_AvSigVersion_num.columns = ['NUM_Versiones']

In [ ]:
df_AvSigVersion_num['AvSigVersion'] = df_AvSigVersion_num.index

In [ ]:
df_AvSigVersion_num.head(2)

In [ ]:
df = df.merge(df_AvSigVersion_num, on = 'AvSigVersion')

In [ ]:
df.drop('AvSigVersion', axis = 1, inplace = True)

In [ ]:
df.head(5)

In [ ]:
df_OsBuildLab_num = pd.DataFrame(df['OsBuildLab'].value_counts(dropna = False))# variable OsBuildLab

In [ ]:
df_OsBuildLab_num.columns = ['NUM_build']

In [ ]:
df_OsBuildLab_num['OsBuildLab'] = df_OsBuildLab_num.index

In [ ]:
df_OsBuildLab_num.head(2)

In [ ]:
df = df.merge(df_OsBuildLab_num, on = 'OsBuildLab')

In [ ]:
df.drop('OsBuildLab', axis = 1, inplace = True)

In [ ]:
df_Census_OSVersion_num = pd.DataFrame(df['Census_OSVersion'].value_counts(dropna = False))# variable Census_OSVersion

In [ ]:
df_Census_OSVersion_num.columns = ['census_versiones']

In [ ]:
df_Census_OSVersion_num['Census_OSVersion'] = df_Census_OSVersion_num.index

In [ ]:
df_Census_OSVersion_num.head(2)

In [ ]:
df = df.merge(df_Census_OSVersion_num, on = 'Census_OSVersion')

In [ ]:
df.drop('Census_OSVersion', axis = 1, inplace = True)

In [ ]:
df.head(2)

**ONE HOT ENCODING**

In [ ]:
df.describe(exclude=np.number).T

In [ ]:
df.info(verbose = True)

In [ ]:
df.drop(['DefaultBrowsersIdentifier', 'AVProductStatesIdentifier', 'CountryIdentifier', 'Census_OEMNameIdentifier', 'Census_OEMModelIdentifier', 'Census_OSUILocaleIdentifier', 'Wdft_RegionIdentifier'], axis = 1, inplace = True)

In [ ]:
def generate_dummies(dataframe, column_name):
    _dummy_dataset = pd.get_dummies(dataframe[column_name], prefix=column_name)
    dataframe = pd.concat([dataframe, _dummy_dataset], axis=1)
    return dataframe.drop(column_name, axis=1)

In [ ]:
for column in df.select_dtypes(exclude=np.number).columns:
    df = generate_dummies(df, column)

In [ ]:
df.info(verbose = False)

###MODELLING

In [ ]:
df['NUM_Versiones'].value_counts()

In [ ]:
df.head(2).T

In [ ]:
df.columns

In [ ]:
df.pivot_table(index = ['NUM_Versiones'], values = TARGET, aggfunc = [len, sum, np.mean])

In [ ]:
dev_df = df[(df['NUM_Versiones'] <= 517)]#development = train + test
val_df = df[(df['NUM_Versiones'] >= 517)]# validation

In [ ]:
df['NUM_Versiones'].describe()

In [ ]:
dev_df.shape

In [ ]:
val_df.shape

Diferenciar entre lo que son atributos y lo que no lo es y se realiza tanto el parte del development y del validation

In [ ]:
dev_df_X = dev_df.drop('HasDetections', axis=1) #--> todos los predictores que tengo 
dev_df_y = dev_df[['HasDetections']] # vector

In [ ]:
val_df_X = val_df.drop('HasDetections', axis=1)
val_df_y = val_df[['HasDetections']]

In [ ]:
dev_df_X.head().T

In [ ]:
dev_df_y.head().T

###Random Holdout : seleccionar entre train y test, esta division es aleatoria

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
                                        dev_df_X, # X 
                                        dev_df_y, # y
                                        test_size = 0.30, # proporcion que queremos para train y test
                                        random_state = 42
                                     )

In [ ]:
X_train.info(verbose=False) # tenemos 175219 filas

In [ ]:
X_test.info(verbose=False) # tenemos 75094 filas 

In [ ]:
X_train.describe().T.head()

In [ ]:
X_test.describe().T.head()

In [ ]:
y_train.describe().T.head()

In [ ]:
y_test.describe().T.head()

###Entrenamiento del modelo

1. **Importar los scikits de la modelizacion**

In [ ]:
!conda install python-graphviz -y

In [ ]:
!conda install pydot -y

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz


from sklearn import metrics
from sklearn import model_selection


2. **Instanciar el modelo**: decidimo aplicar el modelo de decision tree clasifier porque el targuet es categorico y dicotomic. Seleccionamos los parametros de max_depth y random_state.

In [ ]:
dt = DecisionTreeClassifier(
                        # criterion='gini',
                        # splitter='best',
                        max_depth=8, # Importante, regulará el sobreajuste y es el nº de preguntas
                        # min_samples_split=2,
                        # min_samples_leaf=1,
                        # min_weight_fraction_leaf=0.0,
                        # max_features=None,
                        random_state=42,
                        # max_leaf_nodes=None,
                        # min_impurity_decrease=0.0,
                        # min_impurity_split=None,
                        # class_weight=None,
                        # presort=False,  
                )

3.**Entrenar el modelo**

In [ ]:
dt.fit(
    X=X_train, 
    y=y_train, 
    # sample_weight=None, 
    # check_input=True, 
    # X_idx_sorted=None
)

In [ ]:
dot_data = export_graphviz(
                        decision_tree = dt,# important
                        out_file=None, #important
                        # max_depth=None,
                        feature_names=X_test.columns, # important
                        class_names=['No malware', 'Yes malware'], # important
                        # label='all',
                        filled=True,
                        # leaves_parallel=False,
                        impurity=True,
                        # node_ids=False,
                        proportion=True,
                        rotate=True,
                        rounded=True,
                        # special_characters=False,
                        precision=4,
                        )

In [ ]:
 graphviz.Source(dot_data) # generación del arbol

1.4 **Evaluar el modelo**

In [ ]:
y_test_pred = pd.DataFrame(dt.predict(X_test), index = y_test.index, columns =['Malware Prediction']) #amb threshold del 0.5 y evaluar el modelo sobre el test. Mantengo el index del dt del test

In [ ]:
results_df = y_test.join(y_test_pred) # aqui vemos lo que hay con lo que yo he predecido.

In [ ]:
results_df.head(10)

In [ ]:
results_df['succes'] = (results_df['HasDetections'] == results_df['Malware Prediction']).astype(int)

In [ ]:
results_df['succes'].mean()  #Accuracy 

Esto es el Accuracy y podemos ver que el 62,48% són aciertos. Tenemos en cuenta que este dataset está balanceado de modo que se ha fijado tanto en los unos y  los 0 y por ello, creo que el modelo se ajusta bastante a toda la muestra. 

Comprovamos el resultado con la matriz de confusión

In [ ]:
confussion_matrix = pd.crosstab(results_df['HasDetections'], results_df['Malware Prediction'])

In [ ]:
confussion_matrix

Con la matriz podemos revisar si tendemos a la clase mayoriatria o no. En este caso predigo la cantidad de 41514 de 0 y en cambio, 33581 que predigo 1. Vemos con más detalle que el modelo tiende a tener un poco más en cuenta los 0 que los 1 pero de manera leve. 


In [ ]:
TN = confussion_matrix.iloc[0,0]
FP = confussion_matrix.iloc[0,1]
FN = confussion_matrix.iloc[1,0]
TP = confussion_matrix.iloc[1,1]

In [ ]:
accuracy = (TN + TP)/ (TN+TP+FN+FP)# da lo mismo que el accuracy que he fet més a dalt.

In [ ]:
accuracy

In [ ]:
dt.score(X_test, y_test) # --> manera més rapida de fer el accuracy

1.5  **Predicción probabilistica**

In [ ]:
dt.predict_proba(X_test)# nos muestra un array con la probabilidad de 0 y la probabilidad de 1. Me quedo sólo con la probabilidad de uno

In [ ]:
(dt.predict_proba(X_test)[:,1])

Tenir en compte que la probabilitat es del 50% per tant si es major a 0.5 será un 1 i si es inferior serà un 0. Per exemple el primer valor de 0.6852 es un 1 i en canvi, el valor de 0.3521 es un 0. 

In [ ]:
y_score = pd.DataFrame(dt.predict_proba(X_test)[:,1], index = y_test.index, columns = ['Malware Score'])

In [ ]:
y_score

In [ ]:
y_score.nunique() #--> crec que no es correcte, hauria de sortir 8 ja que he demanat 8 preguntes al model. No sé on està l'error.  

In [ ]:
results_df = results_df.join(y_score)

In [ ]:
results_df

**F1 SCORE**

In [ ]:
metrics.f1_score(y_test, y_test_pred)

Té un score del 54%. Es prou significatiu de que el model ajusta. 

**AUC**

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [ ]:
metrics.roc_auc_score(results_df['HasDetections'], results_df['Malware Score'])

El AUC es del 67%, està per sobre del 50% però es un model que ajusta molt poc.

In [ ]:
frp, tpr, th = metrics.roc_curve(results_df['HasDetections'], results_df['Malware Score'])#--> no entenc pq em surt un array tan gran. 

In [ ]:
frp  

In [ ]:
tpr

In [ ]:
th

In [ ]:
plt.plot(frp, tpr)
plt.plot([0,1], [0,1])

**Comprovar si hay overtiffing**

In [ ]:
for i in range(1,15) :
 dt_overfit = DecisionTreeClassifier(max_depth = i, random_state = 42)
 dt_overfit.fit(X_train, y_train)
 acc_test = dt_overfit.score(X_test, y_test)
 acc_train = dt_overfit.score(X_train, y_train)

 print(f'El modelo se entrena con una profundidad de {i} preguntas')
 print(f' La accuracy del modelo en train es {np.round(acc_train,4)}, en test es {np.round(acc_test,4)}')
 print(f' La diferencia es de {np.round(acc_train-acc_test, 4)}')
 print('_________________________________________________________________')

Com més preguntes li fem al model el train augmenta, en canvi el test baixa, de manera que si afegissim més preguntes el train s'adaptaria molt més al model generant overfitting. 

En aquest cas, s'observa que tan el train com el test es mantenent més o menys amb el mateix percentatge, tot i que hi ha una diferencia significativa entre la 7 i 11 pregunta, ja que entre la diferència de train i test comença a augmentar, 
Aleshores, al model, com a màxim seria bo fer-li 11 preguntes per deixar la flexibilitat justa i que no generes l'overfitting. 

